In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("soil1.csv")

In [3]:
df.head()

,Zn %,Fe%,Cu %,Mn %,B %,S %,N,P,K,temperature,humidity,ph,rainfall,label
0,67.67,65.14,91.88,77.70,73.54,85.90,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,80.51,78.19,99.77,91.82,89.04,88.62,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,79.27,88.14,95.54,97.24,88.05,95.67,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,58.30,71.16,98.86,91.40,86.15,86.81,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78.62,82.02,98.05,95.23,65.78,98.56,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [4]:
df['label'].unique()

array(['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas',
       'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate'],
      dtype=object)

In [5]:
n_map = {
    'rice': 100,
    'maize': 125,
    'chickpea': 30,
    'kidneybeans': 30,
    'pigeonpeas': 30,
    'mothbeans': 30,
    'mungbean': 30,
    'blackgram': 30,
    'lentil': 30,
    'pomegranate': 150
}
p_map = {
    'rice': 50,
    'maize': 80,
    'chickpea': 50,
    'kidneybeans': 50,
    'pigeonpeas': 50,
    'mothbeans': 50,
    'mungbean': 50,
    'blackgram': 50,
    'lentil': 50,
    'pomegranate': 75
}
k_map = {
    'rice': 50,
    'maize': 80,
    'chickpea': 25,
    'kidneybeans': 25,
    'pigeonpeas': 25,
    'mothbeans': 25,
    'mungbean': 25,
    'blackgram': 25,
    'lentil': 25,
    'pomegranate': 75
}

In [6]:
df['n_thres'] = df['label'].map(n_map)

In [7]:
df['p_thres'] = df['label'].map(p_map)

In [8]:
df['k_thres'] = df['label'].map(k_map)

In [9]:
df.head()

,Zn %,Fe%,Cu %,Mn %,B %,S %,N,P,K,temperature,humidity,ph,rainfall,label,n_thres,p_thres,k_thres
0,67.67,65.14,91.88,77.70,73.54,85.90,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,100,50,50
1,80.51,78.19,99.77,91.82,89.04,88.62,85,58,41,21.770462,80.319644,7.038096,226.655537,rice,100,50,50
2,79.27,88.14,95.54,97.24,88.05,95.67,60,55,44,23.004459,82.320763,7.840207,263.964248,rice,100,50,50
3,58.30,71.16,98.86,91.40,86.15,86.81,74,35,40,26.491096,80.158363,6.980401,242.864034,rice,100,50,50
4,78.62,82.02,98.05,95.23,65.78,98.56,78,42,42,20.130175,81.604873,7.628473,262.717340,rice,100,50,50


In [10]:
df['n_spray'] = df['n_thres'] - df['N']
df['p_spray'] = df['p_thres'] - df['P']
df['k_spray'] = df['k_thres'] - df['K']

In [17]:
df.head()

,Zn %,Fe%,Cu %,Mn %,B %,S %,N,P,K,temperature,humidity,ph,rainfall,n_thres,p_thres,k_thres,n_spray,p_spray,k_spray
0,67.67,65.14,91.88,77.70,73.54,85.90,90,42,43,20.879744,82.002744,6.502985,202.935536,100,50,50,10,8,7
1,80.51,78.19,99.77,91.82,89.04,88.62,85,58,41,21.770462,80.319644,7.038096,226.655537,100,50,50,15,-8,9
2,79.27,88.14,95.54,97.24,88.05,95.67,60,55,44,23.004459,82.320763,7.840207,263.964248,100,50,50,40,-5,6
3,58.30,71.16,98.86,91.40,86.15,86.81,74,35,40,26.491096,80.158363,6.980401,242.864034,100,50,50,26,15,10
4,78.62,82.02,98.05,95.23,65.78,98.56,78,42,42,20.130175,81.604873,7.628473,262.717340,100,50,50,22,8,8


In [18]:
df.shape

(6730, 19)

In [13]:
df.drop(columns=['label', 'n_thres', 'p_thres', 'k_thres'], inplace=True)

In [21]:
inputs = df.iloc[:, :-3].values
outputs = df.iloc[:, -3:].values

In [22]:
print(inputs.shape, outputs.shape)

(6730, 16) (6730, 3)


In [23]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.2)

input_layer = keras.layers.Input(shape=(16, ))
x = keras.layers.Dense(4096, activation='relu')(input_layer)
x = keras.layers.Dense(4096, activation='relu')(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dense(512, activation='relu')(x)


n_layer = keras.layers.Dense(128, activation='relu')(x)
n_layer = keras.layers.Dense(128, activation='relu')(n_layer)
n_layer = keras.layers.Dense(64, activation='relu')(n_layer)
n_layer = keras.layers.Dense(64, activation='relu')(n_layer)
n_layer_out = keras.layers.Dense(1)(n_layer)


p_layer = keras.layers.Dense(128, activation='relu')(x)
p_layer = keras.layers.Dense(128, activation='relu')(p_layer)
p_layer = keras.layers.Dense(64, activation='relu')(p_layer)
p_layer = keras.layers.Dense(64, activation='relu')(p_layer)
p_layer_out = keras.layers.Dense(1)(p_layer)


k_layer = keras.layers.Dense(128)(x)
k_layer = keras.layers.Dense(128)(k_layer)
k_layer = keras.layers.Dense(64)(k_layer)
k_layer = keras.layers.Dense(64)(k_layer)
k_layer_out = keras.layers.Dense(1)(k_layer)


model = keras.models.Model(inputs=input_layer, outputs=[n_layer_out, p_layer_out, k_layer_out])

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 16)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 4096)                 69632     ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 4096)                 1678131   ['dense[0][0]']               
                                                          2                                       
                                                                                                  
 dense_2 (Dense)             (None, 2048)                 8390656   ['dense_1[0][0]']         

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
history = model.fit(X_train, y_train, epochs=1, batch_size=32, validation_data=(X_test, y_test))

  7/169 [>.............................] - ETA: 2:13 - loss: nan - dense_16_loss: nan - dense_21_loss: nan - dense_26_loss: nan - dense_16_mae: nan - dense_21_mae: nan - dense_26_mae: nan